In [9]:
import numpy as np
from mlxtend.data import loadlocal_mnist
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [2]:
xTrain,yTrain=loadlocal_mnist(images_path='./mnist/train-images.idx3-ubyte', labels_path='./mnist/train-labels.idx1-ubyte')
xTest,yTest = loadlocal_mnist(images_path='./mnist/t10k-images.idx3-ubyte', labels_path='./mnist/t10k-labels.idx1-ubyte')

In [3]:
print(xTrain.shape,yTrain.shape)

(60000, 784) (60000,)


In [4]:
#class for FDA
class FDA:
    W = None
    eigenValues = None
    Sw, Sb = None, None

    def __init__(self):
        W = None
        eigenValues = None
        Sw, Sb = None, None
        
    def fit(self, x, y):
        Sw,Sb = self.getScatters(x, y)
        self.Sw = Sw
        self.Sb = Sb
        Sw_inv_Sb = np.matmul(np.linalg.inv(Sw), Sb)
        eigValues, eigVectors = np.linalg.eigh(Sw_inv_Sb)
        idx = eigValues.argsort()[::-1]   
        eigVectors = eigVectors[:,idx]
        eigValues = eigValues[idx]
        rank = np.linalg.matrix_rank(Sb)
        eigVectors = eigVectors[:,:rank]
        eigValues  = eigValues[:rank]
        self.eigenValues = eigValues
        self.W = eigVectors

    def getScatters(self, x, y):
        Si = []
        uniqueY = np.unique(y)
        for label in uniqueY:
            indexes = np.where(y == label)[0]
            selectedX = x[indexes]
            selectedXTrans = selectedX.T
            scatterMatrix = np.cov(selectedXTrans, ddof=0) * selectedXTrans.shape[1]
            Si.append(scatterMatrix)
        Sw = np.zeros(Si[0].shape)
        for scatterMatrix in Si:
            Sw += scatterMatrix
            
        xTrans = x.T
        St = np.cov(xTrans, ddof=0) * xTrans.shape[1]
        Sb = St - Sw
        
        return Sw,Sb

In [8]:
scaler = StandardScaler()
scaler.fit(xTrain)
pca = PCA(n_components=15)

xTrain = scaler.transform(xTrain)
xTest = scaler.transform(xTest)

pca.fit(xTrain)
xTrain = pca.transform(xTrain)
xTest = pca.transform(xTest)

fda = FDA()
fda.fit(xTrain,yTrain)

W = fda.W
xTrainProjected = np.matmul(W.T,xTrain.T).T
xTestProjected = np.matmul(W.T,xTest.T).T

clf = LinearDiscriminantAnalysis()
clf.fit(xTrainProjected,yTrain)

print("Accuracy we get is - ", clf.score(xTestProjected, yTest)*100,"%")

Accuracy we get is -  79.36999999999999 %


In [11]:
yPredict = clf.predict(xTestProjected)
matrix = confusion_matrix(yTest, yPredict)
accuracyClass = matrix.diagonal()/matrix.sum(axis=1)
for i in range(10):
    print("Accuracy for class "+str(i)+" = ",accuracyClass[i]*100,"%")

Accuracy for class 0 =  86.3265306122449 %
Accuracy for class 1 =  95.94713656387665 %
Accuracy for class 2 =  74.70930232558139 %
Accuracy for class 3 =  79.50495049504951 %
Accuracy for class 4 =  76.17107942973523 %
Accuracy for class 5 =  66.59192825112108 %
Accuracy for class 6 =  83.08977035490605 %
Accuracy for class 7 =  79.47470817120622 %
Accuracy for class 8 =  74.435318275154 %
Accuracy for class 9 =  74.13280475718534 %
